# Домашнее задание


# 🎈🍾 🎉





Вы работаете в крупном онлайн-магазине, который реализует товары «всё для праздников». Вам необходимо провести ABC-анализ и выявить наиболее важные товары на основе объемов продаж и популярности. Эта информация составит план закупок на весь следующий год. 

**ABC-анализ** позволяет определить наиболее важные ресурсы компании на основе объемов продаж и прибыли.

В основе ABC-анализа лежит __«Принцип Парето»: 20% ресурсов приносят 80% прибыли__. Именно этот вид анализа позволяет выделить те самые  группы товаров, ресурсов, клиентов, поставщиков и т. п., которые и приносят компании основную прибыль.

__Задача анализа разделить товары на группы по принципу:__

- А — наиболее ценные, 20% — ассортимента (номенклатура); 80% — продаж
- В — промежуточные, 30% — ассортимента; 15% — продаж
- С — наименее ценные, 50% — ассортимента; 5% — продаж

## Задание 1

Сохраните все данные из таблицы `products` из PostgreSQL в дата-фрейм `products` и подготовьте данные к анализу.

1. Подгрузите данные для подключения
2. Подключите нужную библиотеку для работы с PostgreSQL
3. Создайте соединение с БД
4. Создайте запрос и выгрузите все данные из таблицы `products`
5. Положите данные в дата-фрейм `products`
6. Приведите данные к удобному типу: числа к числовому типу, идентификаторы — к текстовому


In [1]:
from postgresql_secret import pg_secret

In [2]:
pg_secret


{'db_host': '178.154.254.133',
 'db_name': 'skyeng',
 'db_password': 'readonlyuser',
 'db_port': '5432',
 'db_user': 'readonlyuser'}

In [3]:
import psycopg2
# создай соединение
connection = psycopg2.connect(
                            database = pg_secret['db_name'],
                            user = pg_secret['db_user'],
                            password = pg_secret['db_password'],
                            host = pg_secret['db_host'],
                            port = pg_secret['db_port']
                            )
# заведи курсор, чтобы делать запросы в базу
cursor = connection.cursor()

In [4]:
import pandas as pd
# напиши запрос, твоих знаний SQL точно хватит
query = 'SELECT * FROM products'
cursor.execute(query)
result = cursor.fetchall()

columns = []
for desc in cursor.description:
    columns.append(desc[0])

products = pd.DataFrame(result,columns=columns)

products.head()


,invoice_id,stock_code,description,quantity,price
0,568954,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2.10
1,568954,21137,BLACK RECORD COVER FRAME,72,3.39
2,568954,82581,TOILET METAL SIGN,12,0.55
3,568954,21756,BATH BUILDING BLOCK WORD,3,6.25
4,568954,21754,HOME BUILDING BLOCK WORD,3,6.25


In [5]:
products['invoice_id'] = products['invoice_id'].apply(str)
products['price'] = products['price'].astype('float64')
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 779495 entries, 0 to 779494
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   invoice_id   779495 non-null  object 
 1   stock_code   779495 non-null  object 
 2   description  779495 non-null  object 
 3   quantity     779495 non-null  int64  
 4   price        779495 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 29.7+ MB


## Задание 2

__Посчитайте, какой доход принес каждый продукт и сколько единиц каждого продукта было продано__

1. Посчитай общую сумму оплат для каждой строки счета
2. Получи общий доход и количество единиц по товарам
3. Объедини сгруппированные данные в один датафрейм со столбцами: `description`, `stock_code`, `quantity`, `total`

In [6]:
import pandas as pd
products.head()

,invoice_id,stock_code,description,quantity,price
0,568954,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2.10
1,568954,21137,BLACK RECORD COVER FRAME,72,3.39
2,568954,82581,TOILET METAL SIGN,12,0.55
3,568954,21756,BATH BUILDING BLOCK WORD,3,6.25
4,568954,21754,HOME BUILDING BLOCK WORD,3,6.25


In [7]:
products['total'] = products['quantity']*products['price'] #добавляем колонку с информацией о выручке

In [8]:
revenue_by_product = products.groupby('stock_code')['total'].sum().reset_index() # посчитай общих доход по каждому товару

In [9]:
sold_by_product = products.groupby('stock_code')['quantity'].sum().reset_index() # посчитай общее количество проданных единиц по каждому товару

In [10]:
revenue_by_product.info()
# объедини общие данные в одну таблицу

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4631 entries, 0 to 4630
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   stock_code  4631 non-null   object 
 1   total       4631 non-null   float64
dtypes: float64(1), object(1)
memory usage: 72.5+ KB


In [63]:
sold_by_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4631 entries, 0 to 4630
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   stock_code  4631 non-null   object
 1   quantity    4631 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 72.5+ KB


In [64]:
products1=products.copy()

info_sold_products=pd.merge(sold_by_product,revenue_by_product) # объединяем таблицу по количеству и выручке

info_sold_products

,stock_code,quantity,total
0,10002,8479,6638.27
1,10080,303,124.61
2,10109,4,1.68
3,10120,648,136.08
4,10123C,628,226.76
...,...,...,...
4626,PADS,18,0.00
4627,POST,5235,124648.04
4628,SP1002,5,14.75
4629,TEST001,60,225.00


In [65]:
products1=products1[['stock_code','description']]

products1=products1.groupby(products1['stock_code']).sum().reset_index() # оставляем уникальные записи


#descriptions=products1.to_dict()
#descriptions.set_index('stock_code',inplace=True)

In [66]:
products1

,stock_code,description
0,10002,INFLATABLE POLITICAL GLOBE INFLATABLE POLITICA...
1,10080,GROOVY CACTUS INFLATABLEGROOVY CACTUS INFLATAB...
2,10109,BENDY COLOUR PENCILS
3,10120,DOGGY RUBBERDOGGY RUBBERDOGGY RUBBERDOGGY RUBB...
4,10123C,HEARTS WRAPPING TAPE HEARTS WRAPPING TAPE HEAR...
...,...,...
4626,PADS,PADS TO MATCH ALL CUSHIONSPADS TO MATCH ALL CU...
4627,POST,POSTAGEPOSTAGEPOSTAGEPOSTAGEPOSTAGEPOSTAGEPOST...
4628,SP1002,KID'S CHALKBOARD/EASELKID'S CHALKBOARD/EASEL
4629,TEST001,This is a test product.This is a test product....


In [67]:
info_sold_products=pd.merge(info_sold_products,products1) #добавляем информацию по описанию

info_sold_products

,stock_code,quantity,total,description
0,10002,8479,6638.27,INFLATABLE POLITICAL GLOBE INFLATABLE POLITICA...
1,10080,303,124.61,GROOVY CACTUS INFLATABLEGROOVY CACTUS INFLATAB...
2,10109,4,1.68,BENDY COLOUR PENCILS
3,10120,648,136.08,DOGGY RUBBERDOGGY RUBBERDOGGY RUBBERDOGGY RUBB...
4,10123C,628,226.76,HEARTS WRAPPING TAPE HEARTS WRAPPING TAPE HEAR...
...,...,...,...,...
4626,PADS,18,0.00,PADS TO MATCH ALL CUSHIONSPADS TO MATCH ALL CU...
4627,POST,5235,124648.04,POSTAGEPOSTAGEPOSTAGEPOSTAGEPOSTAGEPOSTAGEPOST...
4628,SP1002,5,14.75,KID'S CHALKBOARD/EASELKID'S CHALKBOARD/EASEL
4629,TEST001,60,225.00,This is a test product.This is a test product....


## Задание 3

__Найди нарастающий итог по процентам__

1. Отсортируй датафрейм по доходу в нисходящем порядке 
2. Найди процент дохода каждого товара в общем доходе 
3. Сделай нарастающий итог (running total) по процентам каждого товара, для этого найди в интернете и примени метод для расчета кумулятивной суммы в `pandas`. 


Нарастающий итог — это представление данных, в котором мы видим динамику нарастания или убывания суммы от начала отсчета до конца. Получаем мы эту кумулятивную сумму, прибавлением поочередно каждого следующего элемента к сумме предыдущих.  



In [69]:
# отсортируй дата-фрейм (большие вверху, маленькие внизу)
info_sold_products.sort_values('total',ascending=False,inplace=True)

In [73]:
# найди процент дохода для каждого товара 

info_sold_products['share']=info_sold_products['total']/sum(info_sold_products['total'])*100
#(искомая часть / целое число) * 100

In [74]:
info_sold_products

,stock_code,quantity,total,description,share
1609,22423,24139,277656.25,REGENCY CAKESTAND 3 TIERREGENCY CAKESTAND 3 TI...,1.598040
4080,85123A,91814,247203.36,WHITE HANGING HEART T-LIGHT HOLDERWHITE HANGIN...,1.422769
2726,23843,80995,168469.60,"PAPER CRAFT , LITTLE BIRDIE",0.969620
4058,85099B,93436,167920.64,JUMBO BAG RED RETROSPOTJUMBO BAG RED RETROSPOT...,0.966461
4625,M,9391,151777.67,ManualManualManualManualManualManualManualManu...,0.873550
...,...,...,...,...,...
4630,TEST002,1,1.00,This is a test product.,0.000006
247,20721,1,0.85,OWLS CHARLOTTE BAG,0.000005
3419,84205C,2,0.38,HAPPY BIRTHDAY GINGER CAT CARD,0.000002
2847,35930,1,0.38,PINK HEART CHRISTMAS DECORATION,0.000002


In [83]:
# используй метод pandas для кумулятивной суммы
info_sold_products['running_total'] = info_sold_products['share'].cumsum()


In [84]:
info_sold_products

,stock_code,quantity,total,description,share,running_total
1609,22423,24139,277656.25,REGENCY CAKESTAND 3 TIERREGENCY CAKESTAND 3 TI...,1.598040,1.598040
4080,85123A,91814,247203.36,WHITE HANGING HEART T-LIGHT HOLDERWHITE HANGIN...,1.422769,3.020809
2726,23843,80995,168469.60,"PAPER CRAFT , LITTLE BIRDIE",0.969620,3.990429
4058,85099B,93436,167920.64,JUMBO BAG RED RETROSPOTJUMBO BAG RED RETROSPOT...,0.966461,4.956890
4625,M,9391,151777.67,ManualManualManualManualManualManualManualManu...,0.873550,5.830440
...,...,...,...,...,...,...
4630,TEST002,1,1.00,This is a test product.,0.000006,99.999991
247,20721,1,0.85,OWLS CHARLOTTE BAG,0.000005,99.999996
3419,84205C,2,0.38,HAPPY BIRTHDAY GINGER CAT CARD,0.000002,99.999998
2847,35930,1,0.38,PINK HEART CHRISTMAS DECORATION,0.000002,100.000000


## Задание 4

__Пометь товары метками `'A'`, `'B'`, `'C'` в зависимости от нарастающего итога__

1. Напиши функцию, которая будет помечать товары с нарастающим итогом ниже или равно 80% как группу `'A'`, от 80%-95% — как `'B'` и остальные как `'C'`

In [85]:
def revenue(x):   # допиши функцию
    if x<=80:
        return 'A' 
    elif x<=95:
        return 'B'
    else:
        return 'C'  

# создай новый столбец revenue с помощью функции
info_sold_products['revenue'] = info_sold_products['running_total'].apply(revenue)

In [86]:
info_sold_products

,stock_code,quantity,total,description,share,running_total,revenue
1609,22423,24139,277656.25,REGENCY CAKESTAND 3 TIERREGENCY CAKESTAND 3 TI...,1.598040,1.598040,A
4080,85123A,91814,247203.36,WHITE HANGING HEART T-LIGHT HOLDERWHITE HANGIN...,1.422769,3.020809,A
2726,23843,80995,168469.60,"PAPER CRAFT , LITTLE BIRDIE",0.969620,3.990429,A
4058,85099B,93436,167920.64,JUMBO BAG RED RETROSPOTJUMBO BAG RED RETROSPOT...,0.966461,4.956890,A
4625,M,9391,151777.67,ManualManualManualManualManualManualManualManu...,0.873550,5.830440,A
...,...,...,...,...,...,...,...
4630,TEST002,1,1.00,This is a test product.,0.000006,99.999991,C
247,20721,1,0.85,OWLS CHARLOTTE BAG,0.000005,99.999996,C
3419,84205C,2,0.38,HAPPY BIRTHDAY GINGER CAT CARD,0.000002,99.999998,C
2847,35930,1,0.38,PINK HEART CHRISTMAS DECORATION,0.000002,100.000000,C


## Задание 5

__Найди нарастающий итог по количеству товарных единиц и пометь товары метками `'A'`, `'B'`, `'C'`, отмечая тем самым популярность товара__

1. Отсортируй предыдущий датафрейм с данными о доходе теперь по количеству товаров
2. Найди процент каждого товара в общем количестве
3. Найди нарастающий итог по количеству
4. Напиши функцию с теми же параметрами, что и у дохода
5. Отметь группу

In [87]:
# проведи те же операции, что в двух предыдущих заданиях, для оценки популярности товаров
info_sold_products.sort_values('quantity',ascending=False,inplace=True)

info_sold_products['share_quant']=info_sold_products['quantity']/sum(info_sold_products['quantity'])*100


In [88]:
info_sold_products['running_total_quant'] = info_sold_products['share_quant'].cumsum()

In [89]:
info_sold_products

,stock_code,quantity,total,description,share,running_total,revenue,share_quant,running_total_quant
3410,84077,105185,24098.03,WORLD WAR 2 GLIDERS ASSTD DESIGNSWORLD WAR 2 G...,0.138695,34.446876,A,0.999031,0.999031
4058,85099B,93436,167920.64,JUMBO BAG RED RETROSPOTJUMBO BAG RED RETROSPOT...,0.966461,4.956890,A,0.887441,1.886471
4080,85123A,91814,247203.36,WHITE HANGING HEART T-LIGHT HOLDERWHITE HANGIN...,1.422769,3.020809,A,0.872035,2.758506
601,21212,89850,43390.03,PACK OF 72 RETROSPOT CAKE CASESPACK OF 72 RETR...,0.249730,18.467322,A,0.853381,3.611888
2726,23843,80995,168469.60,"PAPER CRAFT , LITTLE BIRDIE",0.969620,3.990429,A,0.769278,4.381166
...,...,...,...,...,...,...,...,...,...
2907,37438,1,3.75,FUNKY FLOWER COFFEE POT,0.000022,99.999388,C,0.000009,99.999962
4279,90025B,1,3.75,BAROQUE BUTTERFLY EARRINGS RED,0.000022,99.999345,C,0.000009,99.999972
4241,90004A,1,3.75,JADE GREEN PAIR ENAMEL HAIR SLIDES,0.000022,99.999324,C,0.000009,99.999981
2874,35974,1,3.75,S/6 CHRISTMAS TREE PLACE SETTING,0.000022,99.999280,C,0.000009,99.999991


In [92]:
def quantity(x): # допиши функцию
    if x<=80:
        return 'A' 
    elif x<=95:
        return 'B'
    else:
        return 'C'  

# создай новый столбец popularity с помощью функции
info_sold_products['popularity'] = info_sold_products['running_total_quant'].apply(quantity)


In [93]:
info_sold_products.head()

,stock_code,quantity,total,description,share,running_total,revenue,share_quant,running_total_quant,popularity
3410,84077,105185,24098.03,WORLD WAR 2 GLIDERS ASSTD DESIGNSWORLD WAR 2 G...,0.138695,34.446876,A,0.999031,0.999031,A
4058,85099B,93436,167920.64,JUMBO BAG RED RETROSPOTJUMBO BAG RED RETROSPOT...,0.966461,4.956890,A,0.887441,1.886471,A
4080,85123A,91814,247203.36,WHITE HANGING HEART T-LIGHT HOLDERWHITE HANGIN...,1.422769,3.020809,A,0.872035,2.758506,A
601,21212,89850,43390.03,PACK OF 72 RETROSPOT CAKE CASESPACK OF 72 RETR...,0.249730,18.467322,A,0.853381,3.611888,A
2726,23843,80995,168469.60,"PAPER CRAFT , LITTLE BIRDIE",0.969620,3.990429,A,0.769278,4.381166,A


## Задание 6

__Объедини группы и проанализируй__

1. Создай серию `group` с объединенным столбцом отметок популярности и доходности товаров
2. Посчитай количество вхождений каждой сгруппированной пары
3. Какие группы непопулярны и приносят мало дохода?
4. На каких группах товаров нужно сфокусироваться при составлении плана закупок?

In [94]:
# напиши здесь код и комментарии
info_sold_products['group']=info_sold_products['revenue']+info_sold_products['popularity']
info_sold_products

,stock_code,quantity,total,description,share,running_total,revenue,share_quant,running_total_quant,popularity,group
3410,84077,105185,24098.03,WORLD WAR 2 GLIDERS ASSTD DESIGNSWORLD WAR 2 G...,0.138695,34.446876,A,0.999031,0.999031,A,AA
4058,85099B,93436,167920.64,JUMBO BAG RED RETROSPOTJUMBO BAG RED RETROSPOT...,0.966461,4.956890,A,0.887441,1.886471,A,AA
4080,85123A,91814,247203.36,WHITE HANGING HEART T-LIGHT HOLDERWHITE HANGIN...,1.422769,3.020809,A,0.872035,2.758506,A,AA
601,21212,89850,43390.03,PACK OF 72 RETROSPOT CAKE CASESPACK OF 72 RETR...,0.249730,18.467322,A,0.853381,3.611888,A,AA
2726,23843,80995,168469.60,"PAPER CRAFT , LITTLE BIRDIE",0.969620,3.990429,A,0.769278,4.381166,A,AA
...,...,...,...,...,...,...,...,...,...,...,...
2907,37438,1,3.75,FUNKY FLOWER COFFEE POT,0.000022,99.999388,C,0.000009,99.999962,C,CC
4279,90025B,1,3.75,BAROQUE BUTTERFLY EARRINGS RED,0.000022,99.999345,C,0.000009,99.999972,C,CC
4241,90004A,1,3.75,JADE GREEN PAIR ENAMEL HAIR SLIDES,0.000022,99.999324,C,0.000009,99.999981,C,CC
2874,35974,1,3.75,S/6 CHRISTMAS TREE PLACE SETTING,0.000022,99.999280,C,0.000009,99.999991,C,CC


In [95]:
info_sold_products['group'].value_counts() #считаем количество вхождений сгруппированных пар

CC    2041
AA     685
BB     577
BC     377
CB     303
BA     270
AB     248
CA      72
AC      58
Name: group, dtype: int64

<div
  style="
    padding: 24px;
    background-color: #009ee4;
    border-radius: 24px;
    color: white;
    font-family: Arial;
  "
>
  <div style="font-size: 36px; margin-top: 6px; line-height: 16px; font-weight: bold">
    Вывод
  <div style=" font-size: 18px; margin-top: 24px; line-height: 24px">
Наименее популярна и приносит меньше всего дохода группа CC, таких товаров больше всего. Группа АС не популярна, но приносит большой доход, возможно это высокостоящие товары, которые не покупают в больших количествах. Группа АА наиболее популярна и приносит больше всего дохода, думаю на ней и стоит сфокусироваться при составлении плана закупок

      

</div>


## Задание 7

__Выгрузи данные по товарам с метокой по ABC-анализу__

1. Объедини по `stock_code` в одной таблице `ABC-код`, описание товара и `ID `

2. Выгрузи таблицу в `csv` для отправки в отдел закупок

In [98]:
products_purch_dep=info_sold_products[['stock_code','description','group']]
products_purch_dep.to_csv('products_purch_dep.csv',index=False)

# 🎉🎉🎉 Поздравляем, ты великолепен! 